# Análisis Exploratorio de Datos (EDA) - TalentPulse HR Data

Este notebook realiza un análisis exploratorio completo sobre los datos de recursos humanos procesados por el pipeline ETL. Incluye:

- **Análisis descriptivo** de empleados, departamentos y métricas clave
- **Visualizaciones interactivas** para identificar patrones y tendencias
- **Correlaciones** entre variables de desempeño, salario y antigüedad
- **Insights de negocio** para la toma de decisiones en RRHH

**Dataset**: Datos limpios y con features procesados por TalentPulse ETL Pipeline  
**Registros**: 2,000,000 empleados  
**Fecha**: Septiembre 2025

## 1. Configuración e Importación de Librerías

Configuramos el entorno e importamos las librerías necesarias para análisis y visualización.

In [ ]:
# Importaciones principales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings

# Configuración de visualizaciones
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

# Configuración de pandas para mostrar más información
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

# Suprimir advertencias menores
warnings.filterwarnings('ignore')

# Configuración de matplotlib
plt.rcParams['font.size'] = 10
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['xtick.labelsize'] = 9
plt.rcParams['ytick.labelsize'] = 9
plt.rcParams['legend.fontsize'] = 9

print("✅ Librerías importadas correctamente")
print(f"📊 Pandas versión: {pd.__version__}")
print(f"📈 Numpy versión: {np.__version__}")

## 2. Carga de Datos

Cargamos el dataset procesado que incluye tanto los datos limpios como las features generadas por el pipeline ETL.

In [ ]:
# Cargar dataset con features procesadas
try:
    # Intentar cargar datos con features primero
    df_features = pd.read_csv('Data/processed/hr_with_features.csv')
    df = df_features.copy()
    print("✅ Dataset con features cargado exitosamente")
    data_source = "Con features"
except FileNotFoundError:
    # Si no existe, cargar datos limpios básicos
    df = pd.read_csv('Data/processed/clean_hr.csv')
    print("⚠️ Dataset básico cargado (sin features)")
    data_source = "Básico"

# Información general del dataset
print(f"\n📊 Información del dataset ({data_source}):")
print(f"   • Registros: {len(df):,}")
print(f"   • Columnas: {len(df.columns)}")
print(f"   • Memoria: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

# Mostrar primeras filas
print(f"\n🔍 Primeras 3 filas:")
df.head(3)

## 3. Análisis Descriptivo General

Exploramos la estructura y características principales del dataset.

In [ ]:
# Información detallada del DataFrame
print("📋 INFORMACIÓN TÉCNICA DEL DATASET")
print("=" * 50)
df.info()

print("\n" + "=" * 50)
print("📊 ESTADÍSTICAS DESCRIPTIVAS - Variables Numéricas")
print("=" * 50)
numeric_stats = df.describe()
display(numeric_stats.round(2))

print("\n" + "=" * 50)
print("📝 ESTADÍSTICAS DESCRIPTIVAS - Variables Categóricas")
print("=" * 50)
categorical_stats = df.describe(include=['object', 'string'])
if not categorical_stats.empty:
    display(categorical_stats)
else:
    print("No hay variables categóricas en el dataset")

## 4. Análisis de Distribuciones por Departamento

Analizamos la composición de la organización por departamentos y sus características.

In [ ]:
# Análisis por departamento
dept_col = 'departamento' if 'departamento' in df.columns else 'Department'

if dept_col in df.columns:
    # Distribución por departamento
    dept_counts = df[dept_col].value_counts()
    
    # Crear figura con múltiples subgráficos
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('📊 Análisis de Distribución por Departamento', fontsize=16, fontweight='bold')
    
    # 1. Gráfico de barras - Cantidad por departamento
    axes[0,0].bar(range(len(dept_counts)), dept_counts.values, color='skyblue', alpha=0.8)
    axes[0,0].set_title('Empleados por Departamento')
    axes[0,0].set_xticks(range(len(dept_counts)))
    axes[0,0].set_xticklabels(dept_counts.index, rotation=45, ha='right')
    axes[0,0].set_ylabel('Número de Empleados')
    
    # Agregar valores en las barras
    for i, v in enumerate(dept_counts.values):
        axes[0,0].text(i, v + max(dept_counts.values)*0.01, f'{v:,}', 
                      ha='center', va='bottom', fontweight='bold')
    
    # 2. Gráfico de pastel - Porcentaje por departamento
    colors = plt.cm.Set3(np.linspace(0, 1, len(dept_counts)))
    wedges, texts, autotexts = axes[0,1].pie(dept_counts.values, labels=dept_counts.index, 
                                           autopct='%1.1f%%', colors=colors, startangle=90)
    axes[0,1].set_title('Distribución Porcentual por Departamento')
    
    # 3. Top 10 departamentos
    top_10_depts = dept_counts.head(10)
    axes[1,0].barh(range(len(top_10_depts)), top_10_depts.values, color='lightcoral', alpha=0.8)
    axes[1,0].set_title('Top 10 Departamentos')
    axes[1,0].set_yticks(range(len(top_10_depts)))
    axes[1,0].set_yticklabels(top_10_depts.index)
    axes[1,0].set_xlabel('Número de Empleados')
    
    # 4. Estadísticas de departamentos
    axes[1,1].axis('off')
    stats_text = f"""
    📈 ESTADÍSTICAS CLAVE
    
    Total Departamentos: {df[dept_col].nunique()}
    Departamento más grande: {dept_counts.index[0]}
    ({dept_counts.iloc[0]:,} empleados)
    
    Departamento más pequeño: {dept_counts.index[-1]}
    ({dept_counts.iloc[-1]:,} empleados)
    
    Promedio por departamento: {dept_counts.mean():.0f}
    Mediana por departamento: {dept_counts.median():.0f}
    """
    axes[1,1].text(0.1, 0.9, stats_text, transform=axes[1,1].transAxes, 
                   fontsize=11, verticalalignment='top', bbox=dict(boxstyle="round,pad=0.3", 
                   facecolor="lightblue", alpha=0.7))
    
    plt.tight_layout()
    plt.show()
    
    # Tabla resumen
    print("\n📋 RESUMEN POR DEPARTAMENTO:")
    print("-" * 50)
    dept_summary = pd.DataFrame({
        'Empleados': dept_counts,
        'Porcentaje': (dept_counts / len(df) * 100).round(1)
    })
    display(dept_summary.head(10))
    
else:
    print("⚠️ Columna de departamento no encontrada en el dataset")

## 5. Análisis Salarial Detallado

Exploramos las distribuciones salariales y su relación con otras variables organizacionales.

In [ ]:
# Análisis salarial
salary_col = 'salario_inr' if 'salario_inr' in df.columns else 'Salary_INR'

if salary_col in df.columns:
    # Crear figura con análisis salarial
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('💰 Análisis Salarial Completo', fontsize=16, fontweight='bold')
    
    # 1. Histograma de salarios
    axes[0,0].hist(df[salary_col], bins=50, alpha=0.7, color='green', edgecolor='black')
    axes[0,0].set_title('Distribución de Salarios')
    axes[0,0].set_xlabel('Salario (INR)')
    axes[0,0].set_ylabel('Frecuencia')
    axes[0,0].ticklabel_format(style='scientific', axis='x', scilimits=(0,0))
    
    # 2. Boxplot de salarios
    axes[0,1].boxplot(df[salary_col], vert=True, patch_artist=True, 
                     boxprops=dict(facecolor='lightblue', alpha=0.7))
    axes[0,1].set_title('Detección de Outliers Salariales')
    axes[0,1].set_ylabel('Salario (INR)')
    axes[0,1].ticklabel_format(style='scientific', axis='y', scilimits=(0,0))
    
    # 3. Salarios por departamento (top 10)
    if 'departamento' in df.columns or 'Department' in df.columns:
        dept_col = 'departamento' if 'departamento' in df.columns else 'Department'
        salary_by_dept = df.groupby(dept_col)[salary_col].mean().sort_values(ascending=False).head(10)
        
        axes[0,2].barh(range(len(salary_by_dept)), salary_by_dept.values, color='orange', alpha=0.8)
        axes[0,2].set_title('Salario Promedio por Departamento (Top 10)')
        axes[0,2].set_yticks(range(len(salary_by_dept)))
        axes[0,2].set_yticklabels(salary_by_dept.index)
        axes[0,2].set_xlabel('Salario Promedio (INR)')
        axes[0,2].ticklabel_format(style='scientific', axis='x', scilimits=(0,0))
    else:
        axes[0,2].text(0.5, 0.5, 'Datos de departamento\nno disponibles', 
                       ha='center', va='center', transform=axes[0,2].transAxes)
        axes[0,2].set_title('Salarios por Departamento')
    
    # 4. Distribución por cuartiles
    q1 = df[salary_col].quantile(0.25)
    q2 = df[salary_col].quantile(0.50)
    q3 = df[salary_col].quantile(0.75)
    
    quartile_labels = ['Q1 (25%)', 'Q2 (50%)', 'Q3 (75%)', 'Q4 (100%)']
    quartile_counts = [
        (df[salary_col] <= q1).sum(),
        ((df[salary_col] > q1) & (df[salary_col] <= q2)).sum(),
        ((df[salary_col] > q2) & (df[salary_col] <= q3)).sum(),
        (df[salary_col] > q3).sum()
    ]
    
    axes[1,0].pie(quartile_counts, labels=quartile_labels, autopct='%1.1f%%', 
                 colors=['#ff9999', '#66b3ff', '#99ff99', '#ffcc99'], startangle=90)
    axes[1,0].set_title('Distribución por Cuartiles Salariales')
    
    # 5. Tendencia salarial (si hay datos de antigüedad)
    if 'antiguedad_anos' in df.columns:
        # Agrupar por años de antigüedad
        salary_by_tenure = df.groupby(df['antiguedad_anos'].round())[salary_col].mean()
        axes[1,1].plot(salary_by_tenure.index, salary_by_tenure.values, 
                      marker='o', linewidth=2, markersize=6, color='purple')
        axes[1,1].set_title('Salario Promedio vs Antigüedad')
        axes[1,1].set_xlabel('Años de Antigüedad')
        axes[1,1].set_ylabel('Salario Promedio (INR)')
        axes[1,1].grid(True, alpha=0.3)
        axes[1,1].ticklabel_format(style='scientific', axis='y', scilimits=(0,0))
    else:
        axes[1,1].text(0.5, 0.5, 'Datos de antigüedad\nno disponibles', 
                       ha='center', va='center', transform=axes[1,1].transAxes)
        axes[1,1].set_title('Salario vs Antigüedad')
    
    # 6. Estadísticas salariales
    axes[1,2].axis('off')
    salary_stats = f"""
    📊 ESTADÍSTICAS SALARIALES
    
    Promedio: ₹{df[salary_col].mean():,.0f}
    Mediana: ₹{df[salary_col].median():,.0f}
    Mínimo: ₹{df[salary_col].min():,.0f}
    Máximo: ₹{df[salary_col].max():,.0f}
    
    Desviación Estándar: ₹{df[salary_col].std():,.0f}
    
    Cuartiles:
    Q1 (25%): ₹{q1:,.0f}
    Q2 (50%): ₹{q2:,.0f}
    Q3 (75%): ₹{q3:,.0f}
    
    Rango Intercuartil: ₹{q3-q1:,.0f}
    """
    axes[1,2].text(0.1, 0.9, salary_stats, transform=axes[1,2].transAxes, 
                   fontsize=10, verticalalignment='top', 
                   bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgreen", alpha=0.7))
    
    plt.tight_layout()
    plt.show()
    
    # Análisis de outliers salariales
    IQR = q3 - q1
    lower_bound = q1 - 1.5 * IQR
    upper_bound = q3 + 1.5 * IQR
    outliers = df[(df[salary_col] < lower_bound) | (df[salary_col] > upper_bound)]
    
    print(f"\n🔍 ANÁLISIS DE OUTLIERS SALARIALES:")
    print(f"   • Outliers detectados: {len(outliers):,} ({len(outliers)/len(df)*100:.2f}%)")
    print(f"   • Límite inferior: ₹{lower_bound:,.0f}")
    print(f"   • Límite superior: ₹{upper_bound:,.0f}")
    
else:
    print("⚠️ Columna de salario no encontrada en el dataset")

## 6. Análisis de Correlaciones

Exploramos las relaciones entre variables numéricas para identificar patrones y asociaciones importantes en los datos de RR.HH.

In [ ]:
# Seleccionar solo variables numéricas para análisis de correlación
numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()

# Remover columnas de ID si existen
id_columns = ['employee_id', 'Employee_ID', 'ID', 'id']
numeric_columns = [col for col in numeric_columns if col.lower() not in [id_col.lower() for id_col in id_columns]]

print(f"📊 Variables numéricas para análisis de correlación: {len(numeric_columns)}")
print(f"   Variables: {', '.join(numeric_columns)}")

if len(numeric_columns) >= 2:
    # Calcular matriz de correlación
    correlation_matrix = df[numeric_columns].corr()
    
    # Crear figura para visualización de correlaciones
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('🔗 Análisis de Correlaciones entre Variables', fontsize=16, fontweight='bold')
    
    # 1. Mapa de calor completo
    im1 = axes[0,0].imshow(correlation_matrix, cmap='RdBu_r', aspect='auto', vmin=-1, vmax=1)
    axes[0,0].set_title('Matriz de Correlación Completa')
    axes[0,0].set_xticks(range(len(numeric_columns)))
    axes[0,0].set_yticks(range(len(numeric_columns)))
    axes[0,0].set_xticklabels(numeric_columns, rotation=45, ha='right')
    axes[0,0].set_yticklabels(numeric_columns)
    
    # Agregar valores de correlación en el mapa
    for i in range(len(numeric_columns)):
        for j in range(len(numeric_columns)):
            text = axes[0,0].text(j, i, f'{correlation_matrix.iloc[i, j]:.2f}',
                                ha="center", va="center", color="black" if abs(correlation_matrix.iloc[i, j]) < 0.5 else "white",
                                fontsize=8)
    
    plt.colorbar(im1, ax=axes[0,0], fraction=0.046, pad=0.04)
    
    # 2. Mapa de calor con seaborn (más estético)
    sns.heatmap(correlation_matrix, annot=True, cmap='RdBu_r', center=0, 
                square=True, fmt='.2f', cbar_kws={"shrink": .8}, ax=axes[0,1])
    axes[0,1].set_title('Matriz de Correlación (Seaborn)')
    axes[0,1].set_xticklabels(axes[0,1].get_xticklabels(), rotation=45, ha='right')
    
    # 3. Top correlaciones positivas
    # Obtener correlaciones sin la diagonal
    corr_pairs = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            corr_pairs.append({
                'var1': correlation_matrix.columns[i],
                'var2': correlation_matrix.columns[j],
                'correlation': correlation_matrix.iloc[i, j]
            })
    
    # Ordenar por correlación absoluta
    corr_pairs_df = pd.DataFrame(corr_pairs)
    top_positive = corr_pairs_df.nlargest(10, 'correlation')
    
    y_pos = np.arange(len(top_positive))
    axes[1,0].barh(y_pos, top_positive['correlation'], color='green', alpha=0.7)
    axes[1,0].set_yticks(y_pos)
    axes[1,0].set_yticklabels([f"{row['var1']} vs {row['var2']}" for _, row in top_positive.iterrows()], fontsize=8)
    axes[1,0].set_xlabel('Correlación')
    axes[1,0].set_title('Top 10 Correlaciones Positivas')
    axes[1,0].set_xlim(0, 1)
    
    # Agregar valores en las barras
    for i, v in enumerate(top_positive['correlation']):
        axes[1,0].text(v + 0.01, i, f'{v:.3f}', va='center', fontsize=8)
    
    # 4. Top correlaciones negativas
    top_negative = corr_pairs_df.nsmallest(10, 'correlation')
    
    y_pos = np.arange(len(top_negative))
    axes[1,1].barh(y_pos, top_negative['correlation'], color='red', alpha=0.7)
    axes[1,1].set_yticks(y_pos)
    axes[1,1].set_yticklabels([f"{row['var1']} vs {row['var2']}" for _, row in top_negative.iterrows()], fontsize=8)
    axes[1,1].set_xlabel('Correlación')
    axes[1,1].set_title('Top 10 Correlaciones Negativas')
    axes[1,1].set_xlim(-1, 0)
    
    # Agregar valores en las barras
    for i, v in enumerate(top_negative['correlation']):
        axes[1,1].text(v - 0.01, i, f'{v:.3f}', va='center', ha='right', fontsize=8)
    
    plt.tight_layout()
    plt.show()
    
    # Reporte de correlaciones significativas
    print(f"\n🔍 ANÁLISIS DE CORRELACIONES:")
    print(f"\n📈 TOP 5 CORRELACIONES POSITIVAS:")
    for _, row in top_positive.head().iterrows():
        print(f"   • {row['var1']} ↔ {row['var2']}: {row['correlation']:.3f}")
    
    print(f"\n📉 TOP 5 CORRELACIONES NEGATIVAS:")
    for _, row in top_negative.head().iterrows():
        print(f"   • {row['var1']} ↔ {row['var2']}: {row['correlation']:.3f}")
    
    # Identificar correlaciones fuertes (|r| > 0.7)
    strong_correlations = corr_pairs_df[abs(corr_pairs_df['correlation']) > 0.7]
    if not strong_correlations.empty:
        print(f"\n🔥 CORRELACIONES FUERTES (|r| > 0.7):")
        for _, row in strong_correlations.iterrows():
            print(f"   • {row['var1']} ↔ {row['var2']}: {row['correlation']:.3f}")
    else:
        print(f"\n💡 No se encontraron correlaciones fuertes (|r| > 0.7)")

else:
    print("⚠️ Se necesitan al menos 2 variables numéricas para el análisis de correlación")

## 7. Análisis de Performance y Retención

Examinamos los indicadores de desempeño y rotación de personal para identificar factores críticos en la gestión del talento.

In [ ]:
# Análisis de Performance y Retención
performance_cols = ['performance_rating', 'rating_performance', 'Performance_Rating']
attrition_cols = ['attrition', 'renuncia', 'Attrition', 'left_company']

# Identificar columnas de performance
perf_col = None
for col in performance_cols:
    if col in df.columns:
        perf_col = col
        break

# Identificar columnas de attrition/retención
attr_col = None
for col in attrition_cols:
    if col in df.columns:
        attr_col = col
        break

print(f"📊 Análisis de Performance y Retención")
print(f"   • Columna de Performance: {perf_col if perf_col else 'No encontrada'}")
print(f"   • Columna de Attrition: {attr_col if attr_col else 'No encontrada'}")

# Crear figura para análisis
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('📈 Análisis de Performance y Retención de Empleados', fontsize=16, fontweight='bold')

# 1. Distribución de Performance Rating
if perf_col:
    performance_counts = df[perf_col].value_counts().sort_index()
    colors = plt.cm.RdYlGn(np.linspace(0.2, 0.8, len(performance_counts)))
    
    axes[0,0].bar(performance_counts.index, performance_counts.values, color=colors, alpha=0.8, edgecolor='black')
    axes[0,0].set_title('Distribución de Performance Rating')
    axes[0,0].set_xlabel('Performance Rating')
    axes[0,0].set_ylabel('Número de Empleados')
    
    # Agregar porcentajes
    total = performance_counts.sum()
    for i, (rating, count) in enumerate(performance_counts.items()):
        percentage = (count / total) * 100
        axes[0,0].text(rating, count + count*0.01, f'{percentage:.1f}%', 
                      ha='center', va='bottom', fontweight='bold')
    
    # Estadísticas de performance
    perf_stats = f"""
    📊 ESTADÍSTICAS DE PERFORMANCE:
    
    Rating Promedio: {df[perf_col].mean():.2f}
    Mediana: {df[perf_col].median():.2f}
    Moda: {df[perf_col].mode().iloc[0] if not df[perf_col].mode().empty else 'N/A'}
    
    Desviación Estándar: {df[perf_col].std():.2f}
    
    Distribución:
    """
    for rating, count in performance_counts.items():
        perf_stats += f"\n    Rating {rating}: {count:,} ({count/total*100:.1f}%)"
    
    print(f"\n{perf_stats}")
else:
    axes[0,0].text(0.5, 0.5, 'Datos de Performance\nno disponibles', 
                   ha='center', va='center', transform=axes[0,0].transAxes, fontsize=12)
    axes[0,0].set_title('Distribución de Performance Rating')

# 2. Análisis de Attrition/Retención
if attr_col:
    # Convertir a formato estándar si es necesario
    if df[attr_col].dtype == 'object':
        attrition_mapping = {'Yes': 1, 'No': 0, 'yes': 1, 'no': 0, 'YES': 1, 'NO': 0,
                            'True': 1, 'False': 0, 'true': 1, 'false': 0,
                            'Sí': 1, 'No': 0, 'SI': 1, 'NO': 0}
        df[f'{attr_col}_numeric'] = df[attr_col].map(attrition_mapping).fillna(df[attr_col])
        attr_col_numeric = f'{attr_col}_numeric'
    else:
        attr_col_numeric = attr_col
    
    attrition_counts = df[attr_col_numeric].value_counts().sort_index()
    labels = ['Retenido', 'Renuncia'] if len(attrition_counts) == 2 else [f'Categoría {i}' for i in attrition_counts.index]
    colors = ['#2ecc71', '#e74c3c'] if len(attrition_counts) == 2 else plt.cm.Set3(np.linspace(0, 1, len(attrition_counts)))
    
    wedges, texts, autotexts = axes[0,1].pie(attrition_counts.values, labels=labels, autopct='%1.1f%%', 
                                           colors=colors, startangle=90, explode=[0.05]*len(attrition_counts))
    axes[0,1].set_title('Tasa de Retención vs Attrition')
    
    # Mejorar la apariencia del texto
    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontweight('bold')
        autotext.set_fontsize(11)
    
    # Calcular métricas de retención
    total_employees = len(df)
    if len(attrition_counts) == 2:
        retained = attrition_counts.get(0, 0)
        left = attrition_counts.get(1, 0)
        retention_rate = (retained / total_employees) * 100
        attrition_rate = (left / total_employees) * 100
        
        print(f"\n🎯 MÉTRICAS DE RETENCIÓN:")
        print(f"   • Tasa de Retención: {retention_rate:.1f}%")
        print(f"   • Tasa de Attrition: {attrition_rate:.1f}%")
        print(f"   • Empleados Retenidos: {retained:,}")
        print(f"   • Empleados que Renunciaron: {left:,}")
else:
    axes[0,1].text(0.5, 0.5, 'Datos de Attrition\nno disponibles', 
                   ha='center', va='center', transform=axes[0,1].transAxes, fontsize=12)
    axes[0,1].set_title('Análisis de Attrition')

# 3. Performance vs Salario (si ambos están disponibles)
salary_col = 'salario_inr' if 'salario_inr' in df.columns else 'Salary_INR' if 'Salary_INR' in df.columns else None

if perf_col and salary_col:
    # Scatter plot
    scatter = axes[0,2].scatter(df[perf_col], df[salary_col], alpha=0.6, c=df[perf_col], 
                               cmap='viridis', s=30, edgecolors='black', linewidth=0.5)
    axes[0,2].set_title('Performance vs Salario')
    axes[0,2].set_xlabel('Performance Rating')
    axes[0,2].set_ylabel('Salario (INR)')
    axes[0,2].ticklabel_format(style='scientific', axis='y', scilimits=(0,0))
    
    # Agregar línea de tendencia
    z = np.polyfit(df[perf_col].dropna(), df[salary_col].dropna(), 1)
    p = np.poly1d(z)
    axes[0,2].plot(df[perf_col].unique(), p(df[perf_col].unique()), "r--", alpha=0.8, linewidth=2)
    
    plt.colorbar(scatter, ax=axes[0,2], fraction=0.046, pad=0.04)
    
    # Calcular correlación
    correlation = df[perf_col].corr(df[salary_col])
    axes[0,2].text(0.05, 0.95, f'Correlación: {correlation:.3f}', 
                   transform=axes[0,2].transAxes, bbox=dict(boxstyle="round", facecolor='white', alpha=0.8))
else:
    axes[0,2].text(0.5, 0.5, 'Datos insuficientes para\nPerformance vs Salario', 
                   ha='center', va='center', transform=axes[0,2].transAxes, fontsize=12)
    axes[0,2].set_title('Performance vs Salario')

# 4. Performance por Departamento
if perf_col and ('departamento' in df.columns or 'Department' in df.columns):
    dept_col = 'departamento' if 'departamento' in df.columns else 'Department'
    perf_by_dept = df.groupby(dept_col)[perf_col].agg(['mean', 'count']).sort_values('mean', ascending=False)
    
    # Solo mostrar departamentos con más de 10 empleados para relevancia estadística
    perf_by_dept = perf_by_dept[perf_by_dept['count'] >= 10].head(15)
    
    bars = axes[1,0].barh(range(len(perf_by_dept)), perf_by_dept['mean'], 
                         color=plt.cm.RdYlGn(perf_by_dept['mean'] / perf_by_dept['mean'].max()),
                         alpha=0.8, edgecolor='black')
    axes[1,0].set_yticks(range(len(perf_by_dept)))
    axes[1,0].set_yticklabels(perf_by_dept.index, fontsize=9)
    axes[1,0].set_xlabel('Performance Rating Promedio')
    axes[1,0].set_title('Performance Promedio por Departamento')
    axes[1,0].set_xlim(0, df[perf_col].max() * 1.1)
    
    # Agregar valores en las barras
    for i, (bar, value) in enumerate(zip(bars, perf_by_dept['mean'])):
        width = bar.get_width()
        axes[1,0].text(width + 0.01, bar.get_y() + bar.get_height()/2, 
                      f'{value:.2f}', ha='left', va='center', fontsize=8, fontweight='bold')
else:
    axes[1,0].text(0.5, 0.5, 'Datos insuficientes para\nPerformance por Departamento', 
                   ha='center', va='center', transform=axes[1,0].transAxes, fontsize=12)
    axes[1,0].set_title('Performance por Departamento')

# 5. Attrition por Performance
if perf_col and attr_col:
    attrition_by_perf = df.groupby(perf_col)[attr_col_numeric].agg(['mean', 'count'])
    attrition_by_perf['attrition_rate'] = attrition_by_perf['mean'] * 100
    
    bars = axes[1,1].bar(attrition_by_perf.index, attrition_by_perf['attrition_rate'], 
                        color='red', alpha=0.7, edgecolor='black')
    axes[1,1].set_title('Tasa de Attrition por Performance Rating')
    axes[1,1].set_xlabel('Performance Rating')
    axes[1,1].set_ylabel('Tasa de Attrition (%)')
    axes[1,1].set_ylim(0, 100)
    
    # Agregar valores en las barras
    for bar, value in zip(bars, attrition_by_perf['attrition_rate']):
        height = bar.get_height()
        axes[1,1].text(bar.get_x() + bar.get_width()/2., height + 1,
                      f'{value:.1f}%', ha='center', va='bottom', fontweight='bold')
else:
    axes[1,1].text(0.5, 0.5, 'Datos insuficientes para\nAttrition vs Performance', 
                   ha='center', va='center', transform=axes[1,1].transAxes, fontsize=12)
    axes[1,1].set_title('Attrition por Performance Rating')

# 6. Resumen de métricas clave
axes[1,2].axis('off')
summary_text = "📊 RESUMEN EJECUTIVO\n\n"

if perf_col:
    avg_performance = df[perf_col].mean()
    high_performers = (df[perf_col] >= 4).sum() if df[perf_col].max() >= 4 else (df[perf_col] >= df[perf_col].quantile(0.8)).sum()
    total_employees = len(df)
    
    summary_text += f"🎯 PERFORMANCE:\n"
    summary_text += f"   Rating Promedio: {avg_performance:.2f}\n"
    summary_text += f"   Alto Rendimiento: {high_performers:,} ({high_performers/total_employees*100:.1f}%)\n\n"

if attr_col and len(attrition_counts) == 2:
    summary_text += f"🔄 RETENCIÓN:\n"
    summary_text += f"   Tasa Retención: {retention_rate:.1f}%\n"
    summary_text += f"   Tasa Attrition: {attrition_rate:.1f}%\n\n"

if salary_col:
    avg_salary = df[salary_col].mean()
    summary_text += f"💰 COMPENSACIÓN:\n"
    summary_text += f"   Salario Promedio: ₹{avg_salary:,.0f}\n\n"

summary_text += f"👥 TOTAL EMPLEADOS: {len(df):,}"

axes[1,2].text(0.1, 0.9, summary_text, transform=axes[1,2].transAxes, 
               fontsize=11, verticalalignment='top',
               bbox=dict(boxstyle="round,pad=0.5", facecolor="lightblue", alpha=0.7))

plt.tight_layout()
plt.show()

## 8. Insights de Negocio y Recomendaciones

Basándose en el análisis exploratorio, identificamos los hallazgos clave y las recomendaciones estratégicas para la gestión de RR.HH.

In [ ]:
# Generar insights automáticos basados en los datos analizados
print("🎯 TALENT PULSE - INSIGHTS DE NEGOCIO Y RECOMENDACIONES ESTRATÉGICAS")
print("=" * 80)

# 1. Análisis de Diversidad y Composición
print("\n1️⃣ DIVERSIDAD Y COMPOSICIÓN ORGANIZACIONAL:")

# Análisis de género si está disponible
gender_cols = ['genero', 'gender', 'Gender', 'sexo']
gender_col = None
for col in gender_cols:
    if col in df.columns:
        gender_col = col
        break

if gender_col:
    gender_dist = df[gender_col].value_counts(normalize=True) * 100
    print(f"   📊 Distribución por Género:")
    for gender, pct in gender_dist.items():
        print(f"      • {gender}: {pct:.1f}%")
    
    # Recomendación de diversidad
    min_representation = gender_dist.min()
    if min_representation < 30:
        print(f"   ⚠️  ALERTA: Baja representación del género minoritario ({min_representation:.1f}%)")
        print(f"   💡 RECOMENDACIÓN: Implementar estrategias de diversidad e inclusión")

# Análisis departamental
dept_cols = ['departamento', 'Department', 'department']
dept_col = None
for col in dept_cols:
    if col in df.columns:
        dept_col = col
        break

if dept_col:
    dept_size = df[dept_col].value_counts()
    largest_dept = dept_size.index[0]
    largest_pct = (dept_size.iloc[0] / len(df)) * 100
    
    print(f"   🏢 Departamento más grande: {largest_dept} ({largest_pct:.1f}%)")
    if largest_pct > 40:
        print(f"   ⚠️  ALERTA: Concentración alta en un departamento")
        print(f"   💡 RECOMENDACIÓN: Considerar redistribución o crecimiento balanceado")

# 2. Análisis Salarial
print(f"\n2️⃣ ANÁLISIS SALARIAL Y EQUIDAD:")

salary_col = 'salario_inr' if 'salario_inr' in df.columns else 'Salary_INR' if 'Salary_INR' in df.columns else None

if salary_col:
    avg_salary = df[salary_col].mean()
    median_salary = df[salary_col].median()
    salary_std = df[salary_col].std()
    coefficient_variation = (salary_std / avg_salary) * 100
    
    print(f"   💰 Salario Promedio: ₹{avg_salary:,.0f}")
    print(f"   📊 Mediana Salarial: ₹{median_salary:,.0f}")
    print(f"   📈 Coeficiente de Variación: {coefficient_variation:.1f}%")
    
    if coefficient_variation > 50:
        print(f"   ⚠️  ALERTA: Alta variabilidad salarial")
        print(f"   💡 RECOMENDACIÓN: Revisar estructura de compensaciones")
    
    # Análisis de equidad salarial por género
    if gender_col and len(df[gender_col].unique()) == 2:
        salary_by_gender = df.groupby(gender_col)[salary_col].mean()
        gender_gap = abs(salary_by_gender.iloc[0] - salary_by_gender.iloc[1]) / salary_by_gender.max() * 100
        
        print(f"   ⚖️ Brecha Salarial por Género: {gender_gap:.1f}%")
        if gender_gap > 10:
            print(f"   ⚠️  ALERTA: Brecha salarial significativa")
            print(f"   💡 RECOMENDACIÓN: Auditoría de equidad salarial")

# 3. Análisis de Performance
print(f"\n3️⃣ ANÁLISIS DE RENDIMIENTO:")

perf_cols = ['performance_rating', 'rating_performance', 'Performance_Rating']
perf_col = None
for col in perf_cols:
    if col in df.columns:
        perf_col = col
        break

if perf_col:
    avg_performance = df[perf_col].mean()
    perf_std = df[perf_col].std()
    
    # Definir alto rendimiento (top 20% o rating >= 4)
    if df[perf_col].max() >= 4:
        high_performers = (df[perf_col] >= 4).sum()
        low_performers = (df[perf_col] <= 2).sum()
    else:
        high_performers = (df[perf_col] >= df[perf_col].quantile(0.8)).sum()
        low_performers = (df[perf_col] <= df[perf_col].quantile(0.2)).sum()
    
    high_perf_pct = (high_performers / len(df)) * 100
    low_perf_pct = (low_performers / len(df)) * 100
    
    print(f"   🎯 Performance Promedio: {avg_performance:.2f}")
    print(f"   ⭐ Alto Rendimiento: {high_performers:,} empleados ({high_perf_pct:.1f}%)")
    print(f"   ⚠️ Bajo Rendimiento: {low_performers:,} empleados ({low_perf_pct:.1f}%)")
    
    if low_perf_pct > 15:
        print(f"   🚨 ALERTA: Alto porcentaje de bajo rendimiento")
        print(f"   💡 RECOMENDACIÓN: Programa de mejora de performance")

# 4. Análisis de Retención
print(f"\n4️⃣ ANÁLISIS DE RETENCIÓN:")

attr_cols = ['attrition', 'renuncia', 'Attrition', 'left_company']
attr_col = None
for col in attr_cols:
    if col in df.columns:
        attr_col = col
        break

if attr_col:
    # Calcular tasa de attrition
    if df[attr_col].dtype == 'object':
        attrition_rate = (df[attr_col].str.lower().isin(['yes', 'sí', 'true', '1'])).mean() * 100
    else:
        attrition_rate = df[attr_col].mean() * 100
    
    retention_rate = 100 - attrition_rate
    
    print(f"   🔄 Tasa de Retención: {retention_rate:.1f}%")
    print(f"   🚪 Tasa de Attrition: {attrition_rate:.1f}%")
    
    if attrition_rate > 15:
        print(f"   🚨 ALERTA: Alta tasa de rotación")
        print(f"   💡 RECOMENDACIÓN: Investigar causas de rotación")
    elif attrition_rate < 5:
        print(f"   ✅ EXCELENTE: Baja rotación de personal")

# 5. Correlaciones Clave
print(f"\n5️⃣ CORRELACIONES CLAVE:")

numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
id_columns = ['employee_id', 'Employee_ID', 'ID', 'id']
numeric_columns = [col for col in numeric_columns if col.lower() not in [id_col.lower() for id_col in id_columns]]

if len(numeric_columns) >= 2:
    correlation_matrix = df[numeric_columns].corr()
    
    # Encontrar las correlaciones más fuertes
    strong_correlations = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            corr_value = correlation_matrix.iloc[i, j]
            if abs(corr_value) > 0.5:  # Correlaciones moderadas o fuertes
                strong_correlations.append({
                    'var1': correlation_matrix.columns[i],
                    'var2': correlation_matrix.columns[j],
                    'correlation': corr_value
                })
    
    if strong_correlations:
        print(f"   🔗 Correlaciones Significativas Encontradas:")
        for corr in sorted(strong_correlations, key=lambda x: abs(x['correlation']), reverse=True)[:5]:
            direction = "positiva" if corr['correlation'] > 0 else "negativa"
            print(f"      • {corr['var1']} ↔ {corr['var2']}: {corr['correlation']:.3f} ({direction})")

# 6. Recomendaciones Estratégicas
print(f"\n6️⃣ RECOMENDACIONES ESTRATÉGICAS:")

recommendations = []

# Recomendaciones basadas en size
total_employees = len(df)
if total_employees < 100:
    recommendations.append("📈 Considerar plan de crecimiento organizacional estructurado")
elif total_employees > 10000:
    recommendations.append("🏗️ Implementar sistemas de gestión escalables")

# Recomendaciones de datos
missing_data_pct = df.isnull().sum().sum() / (len(df) * len(df.columns)) * 100
if missing_data_pct > 10:
    recommendations.append("🔍 Mejorar la calidad y completitud de los datos de RR.HH.")

# Recomendaciones específicas
if salary_col and coefficient_variation > 50:
    recommendations.append("💰 Establecer bandas salariales más estructuradas")

if perf_col and low_perf_pct > 15:
    recommendations.append("🎯 Implementar programa de desarrollo de performance")

if attr_col and attrition_rate > 15:
    recommendations.append("🔄 Desarrollar estrategia de retención de talento")

# Recomendaciones generales
recommendations.extend([
    "📊 Implementar dashboards de métricas de RR.HH. en tiempo real",
    "🤖 Considerar modelos predictivos para attrition y performance",
    "📋 Establecer KPIs regulares de seguimiento organizacional",
    "🎓 Desarrollar programas de carrera y desarrollo profesional"
])

for i, rec in enumerate(recommendations, 1):
    print(f"   {i}. {rec}")

print(f"\n" + "="*80)
print(f"📈 RESUMEN EJECUTIVO: Dataset analizado con {len(df):,} empleados")
print(f"🔍 Análisis completado con {len(df.columns)} variables")
print(f"📅 Timestamp del análisis: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"="*80)

## 9. Conclusiones y Próximos Pasos

Este análisis exploratorio proporciona una base sólida para la toma de decisiones estratégicas en gestión de talento y desarrollo organizacional.

In [ ]:
# Conclusiones finales del análisis
print("🎯 TALENT PULSE - CONCLUSIONES Y PRÓXIMOS PASOS")
print("=" * 70)

# Resumen de lo que se analizó
print("\n✅ ANÁLISIS COMPLETADO:")
print("   1. Carga y exploración inicial de datos")
print("   2. Análisis estadístico descriptivo")
print("   3. Análisis departamental y organizacional")
print("   4. Evaluación salarial y equidad")
print("   5. Análisis de correlaciones")
print("   6. Evaluación de performance y retención")
print("   7. Generación de insights de negocio")

# Métricas clave del dataset
print(f"\n📊 MÉTRICAS CLAVE DEL DATASET:")
print(f"   • Total de empleados analizados: {len(df):,}")
print(f"   • Variables disponibles: {len(df.columns)}")
print(f"   • Completitud de datos: {((1 - df.isnull().sum().sum() / (len(df) * len(df.columns))) * 100):.1f}%")

# Variables principales identificadas
numeric_vars = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_vars = df.select_dtypes(include=['object']).columns.tolist()

print(f"   • Variables numéricas: {len(numeric_vars)}")
print(f"   • Variables categóricas: {len(categorical_vars)}")

# Próximos pasos recomendados
print(f"\n🚀 PRÓXIMOS PASOS RECOMENDADOS:")

print(f"\n   📈 ANÁLISIS AVANZADO:")
print(f"   1. Segmentación de empleados (clustering)")
print(f"   2. Análisis de series temporales (si hay datos históricos)")
print(f"   3. Análisis de supervivencia para retención")
print(f"   4. Análisis de redes organizacionales")

print(f"\n   🤖 MODELOS PREDICTIVOS:")
print(f"   1. Predicción de attrition/renuncia")
print(f"   2. Predicción de performance rating")
print(f"   3. Optimización salarial")
print(f"   4. Identificación de empleados en riesgo")

print(f"\n   📊 DASHBOARD Y REPORTING:")
print(f"   1. Dashboard ejecutivo interactivo")
print(f"   2. Reportes automáticos mensuales")
print(f"   3. Alertas de métricas críticas")
print(f"   4. Análisis comparativo temporal")

print(f"\n   🎯 ACCIONES OPERATIVAS:")
print(f"   1. Definir KPIs de RR.HH.")
print(f"   2. Establecer benchmarks internos")
print(f"   3. Crear planes de acción específicos")
print(f"   4. Implementar sistema de seguimiento")

# Valor del análisis
print(f"\n💡 VALOR GENERADO POR ESTE ANÁLISIS:")
print(f"   • Comprensión profunda de la composición organizacional")
print(f"   • Identificación de oportunidades de mejora")
print(f"   • Base de datos para decisiones estratégicas")
print(f"   • Framework para análisis futuros")
print(f"   • Detección de riesgos y alertas tempranas")

# Contacto y soporte
print(f"\n📧 SOPORTE TÉCNICO:")
print(f"   • Para dudas sobre el análisis: equipo-analytics@empresa.com")
print(f"   • Para nuevos requerimientos: rrhh-data@empresa.com")
print(f"   • Documentación completa: /docs/analytics/hr-dashboard")

print(f"\n" + "="*70)
print(f"🏆 ANÁLISIS COMPLETADO EXITOSAMENTE")
print(f"📅 Fecha: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"👨‍💼 Analista: TalentPulse Analytics Engine")
print(f"🔄 Versión: 1.0")
print(f"="*70)

# Guardar resumen en archivo de texto (opcional)
try:
    summary_path = "../reports/eda_summary_report.txt"
    with open(summary_path, 'w', encoding='utf-8') as f:
        f.write(f"TALENT PULSE - REPORTE DE ANÁLISIS EXPLORATORIO\n")
        f.write(f"{'='*60}\n\n")
        f.write(f"Fecha del análisis: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Dataset analizado: {len(df):,} empleados\n")
        f.write(f"Variables analizadas: {len(df.columns)}\n\n")
        f.write("Este análisis proporcionó insights clave para la gestión estratégica de RR.HH.\n")
        f.write("Consulte el notebook completo para detalles específicos y visualizaciones.\n")
    print(f"\n📁 Resumen guardado en: {summary_path}")
except Exception as e:
    print(f"\n⚠️ No se pudo guardar el resumen: {str(e)}")

print(f"\n🎉 ¡Gracias por usar TalentPulse Analytics!")